In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_csv('ph.csv', compression='gzip')

/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
d.head()

,id,creation_date,post_id,post_history_type_id,revision_guid,user_id,text,comment
0,28369829,2012-08-24 08:18:33.29 UTC,12105586,1,95c45c9a-aff1-4b7c-adc2-76c727c6a930,1165423.0,Loading String Resources from A Different Path,NaN
1,95514764,2015-07-20 18:28:11.287 UTC,31523754,1,c8ef2715-59bb-429b-ad39-25086ec4e123,5136201.0,System.Data.SqlClient.SqlException: Incorrect ...,NaN
2,187966461,2018-12-17 09:16:42.123 UTC,53812067,1,72e5ae09-4edc-44f2-ae53-8eacda39a9f4,2105218.0,Do multiple jdbc input run simultaneously or s...,NaN
3,8155373,2010-10-07 21:33:26.907 UTC,3886039,1,f419429b-9756-43ff-baf4-5927f589c73c,56621.0,"""Test with debugger"" in ReSharper?",NaN
4,40793970,2013-05-17 16:18:08.577 UTC,16613724,1,ef698524-da6f-4a1f-b0ba-310b38e730d7,2394636.0,jQuerymobile: Close a popup when opening a panel,NaN


In [71]:
# demo from online:
import string
import math

tokenize = lambda doc: doc.lower().split(" ")

document_0 = "China has a strong economy that is growing at a rapid pace. However politically it differs greatly from the US Economy."
document_1 = "At last, China seems serious about confronting an endemic problem: domestic violence and corruption."
document_2 = "Japan's prime minister, Shinzo Abe, is working towards healing the economic turmoil in his own country for his view on the future of his people."
document_3 = "Vladimir Putin is working hard to fix the economy in Russia as the Ruble has tumbled."
document_4 = "What's the future of Abenomics? We asked Shinzo Abe for his views"
document_5 = "Obama has eased sanctions on Cuba while accelerating those against the Russian Economy, even as the Ruble's value falls almost daily."
document_6 = "Vladimir Putin was found to be riding a horse, again, without a shirt on while hunting deer. Vladimir Putin always seems so serious about things - even riding horses."

all_documents = [document_0, document_1, document_2, document_3, document_4, document_5, document_6]

tokenized_documents = [tokenize(d) for d in all_documents] # tokenized docs
all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
print(tokenized_documents)

[['china', 'has', 'a', 'strong', 'economy', 'that', 'is', 'growing', 'at', 'a', 'rapid', 'pace.', 'however', 'politically', 'it', 'differs', 'greatly', 'from', 'the', 'us', 'economy.'], ['at', 'last,', 'china', 'seems', 'serious', 'about', 'confronting', 'an', 'endemic', 'problem:', 'domestic', 'violence', 'and', 'corruption.'], ["japan's", 'prime', 'minister,', 'shinzo', 'abe,', 'is', 'working', 'towards', 'healing', 'the', 'economic', 'turmoil', 'in', 'his', 'own', 'country', 'for', 'his', 'view', 'on', 'the', 'future', 'of', 'his', 'people.'], ['vladimir', 'putin', 'is', 'working', 'hard', 'to', 'fix', 'the', 'economy', 'in', 'russia', 'as', 'the', 'ruble', 'has', 'tumbled.'], ["what's", 'the', 'future', 'of', 'abenomics?', 'we', 'asked', 'shinzo', 'abe', 'for', 'his', 'views'], ['obama', 'has', 'eased', 'sanctions', 'on', 'cuba', 'while', 'accelerating', 'those', 'against', 'the', 'russian', 'economy,', 'even', 'as', 'the', "ruble's", 'value', 'falls', 'almost', 'daily.'], ['vladim

In [72]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [73]:
%timeit a = jaccard_similarity(tokenized_documents[2],tokenized_documents[4])

4.22 µs ± 89.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
# term-frequency inverse document frequency
def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
#     print(tokenized_document.count(term))
#     print(tokenized_document, term)
    try:
        return 1 + math.log(tokenized_document.count(term))
    except ValueError:
        return 0
    

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

In [75]:
idf_values = inverse_document_frequencies(tokenized_documents)
print(idf_values['abenomics?'])
# 2.94591014906
print(idf_values['the'])

2.9459101490553135
1.336472236621213


In [76]:
def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

In [77]:
%time tfidf_representation = tfidf(all_documents)
# print(tfidf_representation[0], document_0)
# doc vector and document

CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.56 ms


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)

sklearn_representation = sklearn_tfidf.fit_transform(all_documents)
print(tfidf_representation[0])
print(sklearn_representation.toarray()[0].tolist())
print(document_0)

[2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.336472236621213, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 3.8142592685777856, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 2.9459101490553135, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8472978603872037, 2.9459101490553135, 0.0, 0.0, 0.0, 2.9459101490553135, 0.0, 1.8472978603872037, 0.0, 2.252762968495368, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.31019096605521496, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18320378146489946, 0.0, 0.18320378146489946, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23957330918096045, 0.0, 0.0, 0.0, 0.183203

In [49]:
d['text'][1]

"System.Data.SqlClient.SqlException: Incorrect syntax near the keyword 'FROM'"

In [7]:
import pandas as pd
d2 = pd.read_csv('posts_questions.csv', compression='gzip')

/home/neal/miniconda3/envs/keras/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
def clean(line):
#     print(type(line))
    line = line.lower().replace("\n"," ").replace("\r","").replace(',',"").replace(">","> ")
    return line
# clean(d2['body'][0])
# d2.to_csv('pq2.csv', sep='~')
# col = []
# for text in d2['body']:
#     col.append(clean(text))
# d2['cleaned'] = col
# print(d2['cleaned'][0])
# d2['cleaned'].to_csv('pq_clean.csv')
import pandas as pd
d2 = pd.read_csv("pq_clean.csv", names=['index','body'])
# print(d2['body'][0])

In [1]:
# d.head()
# d = 5
def word_count(row):
    return len(row.split(" "))
#     print(row)
#     l = row.split(',')[2]
#     print(l)
#     return len(l.split(" "))

from pyspark import SparkContext
def process(filename):
    sc = SparkContext(appName="Text Process").getOrCreate()
    data = sc.textFile(filename)
#     sc.addPyFile("mfun.pyx")
    scores = sc.parallelize(data).map(mfun.word_count).collect()
    sc.stop()
    return scores

In [5]:
import random
print(random.randint(0,1000))

285


In [8]:
# print(d2[:10])
# sc.stop()
# d3 = d2[:100]
# d3.to_csv('pq.csv')
# from pyspark import SparkContext
# sc = SparkContext(appName = 'TextProcess').getOrCreate()
# def line(a):
#     sp = a.split(",")
#     return a.split("~")
# process('pq.csv')
def word_count(row):
    return len(row.split(" "))

files = sc.textFile("pq_clean.csv") \
.map(lambda line: line.split(",")[1]) \
.cache()

import random
sample = files.take(random.randint(0,10))
print(sample)
# .map(word_count) \
# .cache()
# .sum()
#     .map(lambda line: line.split(",")) \
#     .collect()

["<p> i am trying to create a report to display a summary of the values of the columns for each row.   a basic analogy would an inventory listing.  say i have about 15 locations like 2a 2b 2c 3a 3b 3c etc.   each location has a variety of items and the items each have a specific set of common descriptions i.e. a rating of 1-9 boolean y or n another boolean y or n.  it looks something like this:</p>   <pre> <code> 2a   4       y       n 2a   5       y       y 2a   5       n       y 2a   6       n       n       ... 2b   4       n       y   2b   4       y       y       ...etc. </code> </pre>   <p> what i would like to produce is a list of locations and summary counts of each attribute:</p>   <pre> <code> location    1 2 3 4 5 6 7 8 9      y  n        y n      total 2a                1 2 1            2  2        2 2        4 2b                2                1  1        2          2 ... ___________________________________________________________ totals            3 2 1            3  3    

In [2]:
from pyspark.mllib.feature import HashingTF, IDF

hashingTF = HashingTF()
tf = hashingTF.transform(files)
tf.cache()

PythonRDD[3] at RDD at PythonRDD.scala:53

In [24]:
dir(tf)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bypass_serializer',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_is_barrier',
 '_is_pipelinable',
 '_jrdd',
 '_jrdd_deserializer',
 '_jrdd_val',
 '_memory_limit',
 '_pickled',
 '_prev_jrdd',
 '_prev_jrdd_deserializer',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'barrier',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'fl

In [3]:
# tf.take(1)
%time idf = IDF().fit(tf)
%time tfidf = idf.transform(tf)

CPU times: user 22.7 ms, sys: 5.14 ms, total: 27.9 ms
Wall time: 3min 50s
CPU times: user 5.95 ms, sys: 248 µs, total: 6.19 ms
Wall time: 60.7 ms


In [46]:
# a = tfidf.collect()
# print(a[0])

(1048576,[7357,27527,35920,40200,40649,105642,173606,186435,211440,214770,238153,252894,265159,274911,279832,292370,295235,296409,335453,372567,388504,433442,437751,469732,485029,493959,518279,584844,681187,702216,702740,734443,777769,784206,785264,840959,875351,877522,891534,897367,945367,968035],[3.8226116235622163,14.417285837939875,0.2262768335616402,0.02661803847734302,1.3978806171397122,0.48458119188022175,0.02610314563040646,0.9105784134421238,0.0133458282023745,2.939712438456726,0.009286160842474996,0.0,0.004363024528283841,0.7838649809652338,0.7153792751540569,1.2890116836652659,4.22106448003389,0.0016099999855304493,0.08281624648565256,0.2344022636881407,0.059679926569264526,1.0633573220846997,0.29617030641072484,0.5790244947824792,6.175301282732599,0.0,0.5560562241666805,8.122475324564101e-05,1.3219377634221123,0.0045081045813046925,0.07765881333741419,0.005384157928520484,0.13329220058346902,58.9201038318625,6.277094103388847,0.3649521630589995,0.0,4.752039326253392,0.04381

In [7]:
# look at one file
candidate = clean(d2['body'][4])
# print(candidate)
candidateTf = hashingTF.transform(candidate)
candidateTfIdf = idf.transform(candidateTf)
similarities = tfidf.map(lambda v: v.dot(candidateTfIdf) / (v.norm(2) * candidateTfIdf.norm(2)))
for i in enumerate(similarities.collect()):
    print(i)

%time topFive = sorted(enumerate(similarities.collect()), key= lambda kv: kv[1])[0:5]
# topFive = sorted(enumerate(similarities.collect()), key=lambda (k, v): -v)[0:5]
for idx, val in topFive:
    print("doc '%s' has score %.4f" % (d2['body'][idx], val))

In [36]:
a = similarities.collect()
# print(min(a))

0.0


In [18]:
# print(max(a))
# print(candidateTf.shape)
print(hashingTF.numFeatures)

1048576


In [19]:
# df = pd.read_csv("pq_clean.csv")
# d2.head()

,index,body
0,0,<p> i am trying to create a report to display ...
1,1,<p> i understand payments are a tricky thing b...
2,2,<p> i am making a class to store a reference t...
3,3,<p> i am trying to compare performance of <cod...
4,4,"<p> when i am reading the code of <a href=""htt..."


In [78]:
%time tfidf_representation = tfidf(df)

CPU times: user 4.29 ms, sys: 12 µs, total: 4.31 ms
Wall time: 4.22 ms


In [90]:
# print(tfidf_representation[1])

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
# tokenize = lambda doc: doc.lower().split(" ")
# sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)#, tokenizer=tokenize)
# %time sklearn_representation = sklearn_tfidf.fit_transform(d2['body'])

In [51]:
hashidf = HashingVectorizer()
skh = hashidf.fit_transform(d2['body'])


In [52]:
print(skh.shape)

(344722, 1048576)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37560)
Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func()

In [21]:
# d2.head()
# print(sklearn_tfidf.get_stop_words())
# %time sklearn_tfidf.transform(d2['body'][:1])
import numpy as np
print(sklearn_representation.shape)
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude = np.linalg.norm(vector1)*np.linalg.norm(vector2)
    if not magnitude:
        return 0
    return dot_product/magnitude

(344722, 1517363)


In [80]:
# print(np.sqrt(np.sum(candidate**2)))
print(np.linalg.norm(candidate))

1.0000000000000002
1.0000000000000002


In [30]:
candidate = sklearn_tfidf.transform(d2['body'][:1]).toarray()[0]
target = sklearn_representation[3].toarray()
print(candidate[0].shape, target[0].shape)
# print()
sim = []

# a = np.dot(candidate[0], target[0])
# %time a = cosine_similarity(candidate[0], target[0])
# print(a)
# print(a)

# print(candidate.shape, sklearn_representation[3].shape)
a = cosine_similarity(candidate, sklearn_representation[2].toarray()[0])
# print(a)
# print(a)
# %time np.dot(candidate.T, sklearn_representation[2])
# for count, i in enumerate(sklearn_representation[:1000]):
#     sim.append([cosine_similarity(candidate, i.toarray()[0]), count])
# # print(sim)
# print(sorted(sim)[:5])

def compare(candidate, n):
    sim = []
    for count, i in enumerate(sklearn_representation[:n]):
        sim.append([cosine_similarity(candidate, i.toarray()[0]), count])
    return sorted(sim, reverse=True)[:5]
# print(len(df))
# print(sklearn_representation[1])

() (1517363,)


In [31]:
%time compare(candidate, 10000) # 1.5s for 100, 12.8s for 1000

CPU times: user 1min 26s, sys: 1.56 s, total: 1min 27s
Wall time: 44 s


[[1.0000000000000002, 0],
 [0.15263288080462353, 2233],
 [0.13922132245710933, 2108],
 [0.11908126026567464, 4772],
 [0.11846256923066482, 2416]]

In [35]:
# print(d2['body'][2233])
# print(" ")
# print(d2['body'][0])

<p> i am new in using sql with delphi. i have a table i.e.:</p>   <pre> <code> columnname1 columnname2 columnname3 value 1a    value2a     value3a value 1b    value2b     value3b value 1c    value2c     value3c </code> </pre>   <p> and so on. i have the table displayed on a <code> dbgrid</code>  so currently the adodataset is all connected ok. however i have a small loop</p>   <pre> <code> for i := 0 to adodataset.fieldcount - 1 do begin   writeln(wolfileadodataset.fields[i].asstring); end; </code> </pre>   <p> that writes to a text file. this text file does not have the column names and lists the fields one per line.</p>   <pre> <code> value 1a value 2a value 3a value 1b value 2b value 3b </code> </pre>   <p> etc. how can i get it to list in a row format similar to how i've represented the complete table above?</p> 
 
<p> i am trying to create a report to display a summary of the values of the columns for each row.   a basic analogy would an inventory listing.  say i have about 15 loc

In [59]:
# print(sklearn_representation[0])
# print(candidate)
# for i in candidate:
# import scipy.sparse.csr_matrix
# from scipy.sparse.csr_matrix import todense
a = candidate.toarray()
# print(a)

[[0.06933385 0.         0.         ... 0.         0.         0.        ]]


In [29]:
print(sorted([0,2,3,6,4,8,3]))

[0, 2, 3, 3, 4, 6, 8]
